# The Battle of Neighborhoods - Code
### Applied Data Science Capstone by IBM/Coursera

#### Importing needed libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Using Nominatim to obtain coordinates of cities

In [4]:
address = 'Los Angeles'

geolocator = Nominatim(user_agent="foursquare_agent")
la_location = geolocator.geocode(address)
la_latitude = la_location.latitude
la_longitude = la_location.longitude
print('The geograpical coordinate of Los Angeles are {}, {}.'.format(la_latitude, la_longitude))

The geograpical coordinate of Los Angeles are 34.0536909, -118.2427666.


In [5]:
address = 'San Francisco'

geolocator = Nominatim(user_agent="foursquare_agent")
sf_location = geolocator.geocode(address)
sf_latitude = sf_location.latitude
sf_longitude = sf_location.longitude
print('The geograpical coordinate of San Francisco are {}, {}.'.format(sf_latitude, sf_longitude))

The geograpical coordinate of San Francisco are 37.7790262, -122.4199061.


In [6]:
address = 'Portland'

geolocator = Nominatim(user_agent="foursquare_agent")
por_location = geolocator.geocode(address)
por_latitude = por_location.latitude
por_longitude = por_location.longitude
print('The geograpical coordinate of Portland are {}, {}.'.format(por_latitude, por_longitude))

The geograpical coordinate of Portland are 45.5202471, -122.6741949.


In [7]:
address = 'Seattle'

geolocator = Nominatim(user_agent="foursquare_agent")
sea_location = geolocator.geocode(address)
sea_latitude = sea_location.latitude
sea_longitude = sea_location.longitude
print('The geograpical coordinate of Seattle are {}, {}.'.format(sea_latitude, sea_longitude))

The geograpical coordinate of Seattle are 47.6038321, -122.3300624.


#### Define Foursquare Credentials, Version, Limit and Radius

In [8]:
CLIENT_ID = 'F0QECAXOFELPTLKYCS0AU1NWT2DVQFRSUOG04VX2ZBE3ZL2L' # your Foursquare ID
CLIENT_SECRET = 'EGGXFSJKZU0KMPVH504GQM4KRTAXPMKC4ZJZRT2YATJKJ0IU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 250 # Get top 250 venues
radius = 100000 # Define a radius of 100 kilometers/62 miles


print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: F0QECAXOFELPTLKYCS0AU1NWT2DVQFRSUOG04VX2ZBE3ZL2L
CLIENT_SECRET:EGGXFSJKZU0KMPVH504GQM4KRTAXPMKC4ZJZRT2YATJKJ0IU


### Get the top 100 venues that are in each city within a radius of 100 km/62 miles

In [9]:
la_url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    la_latitude, 
    la_longitude, 
    radius, 
    LIMIT)

la_results = requests.get(la_url).json()

In [10]:
sf_url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    sf_latitude, 
    sf_longitude, 
    radius, 
    LIMIT)

sf_results = requests.get(sf_url).json()

In [11]:
por_url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    por_latitude, 
    por_longitude, 
    radius, 
    LIMIT)

por_results = requests.get(por_url).json()

In [12]:
sea_url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    sea_latitude, 
    sea_longitude, 
    radius, 
    LIMIT)

sea_results = requests.get(sea_url).json()

In [13]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
la_venues = la_results['response']['groups'][0]['items']
    
la_nearby_venues = json_normalize(la_venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng','venue.location.distance']
la_nearby_venues = la_nearby_venues.loc[:, filtered_columns]

# filter the category for each row
la_nearby_venues['venue.categories'] = la_nearby_venues.apply(get_category_type, axis=1)

# clean columns
la_nearby_venues.columns = [col.split(".")[-1] for col in la_nearby_venues.columns]

la_nearby_venues.head(15)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng,distance
0,Walt Disney Concert Hall,Concert Hall,34.055511,-118.249284,634
1,The Broad,Art Museum,34.054474,-118.250051,677
2,The Last Bookstore,Bookstore,34.047620,-118.249852,940
3,Hauser & Wirth,Art Gallery,34.046095,-118.234801,1120
4,Salt & Straw,Ice Cream Shop,34.046065,-118.235473,1083
5,Mr. Speedy Plumbing & Rooter Inc.,Construction & Landscaping,34.042538,-118.233864,1488
6,Los Angeles Music Center,Music Venue,34.057133,-118.248354,642
7,KazuNori: The Original Hand Roll Bar,Sushi Restaurant,34.047716,-118.247452,793
8,Vista Hermosa Park,Park,34.061601,-118.256857,1569
9,Guisados,Taco Place,34.070262,-118.250437,1975


In [15]:
sf_venues = sf_results['response']['groups'][0]['items']
    
sf_nearby_venues = json_normalize(sf_venues) # flatten JSON

# filter columns
sf_nearby_venues = sf_nearby_venues.loc[:, filtered_columns]

# filter the category for each row
sf_nearby_venues['venue.categories'] = sf_nearby_venues.apply(get_category_type, axis=1)

# clean columns
sf_nearby_venues.columns = [col.split(".")[-1] for col in sf_nearby_venues.columns]

sf_nearby_venues.head(15)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng,distance
0,Asian Art Museum,Art Museum,37.780178,-122.416505,325
1,Salt & Straw,Ice Cream Shop,37.776532,-122.426051,607
2,Birba,Wine Bar,37.777750,-122.424159,400
3,Vive La Tarte,Café,37.777012,-122.410899,823
4,The Olympic Club,Gym / Fitness Center,37.788181,-122.411067,1281
5,Ritual Coffee Roasters,Coffee Shop,37.776476,-122.424281,478
6,Collis P. Huntington Park,Park,37.792162,-122.412154,1613
7,The Secret Alley,Performing Arts Venue,37.763764,-122.419352,1699
8,Good Vibrations,Adult Boutique,37.763195,-122.421735,1769
9,Fatted Calf,Butcher,37.775935,-122.423146,446


In [16]:
por_venues = por_results['response']['groups'][0]['items']
    
por_nearby_venues = json_normalize(por_venues) # flatten JSON

# filter columns
por_nearby_venues = por_nearby_venues.loc[:, filtered_columns]

# filter the category for each row
por_nearby_venues['venue.categories'] = por_nearby_venues.apply(get_category_type, axis=1)

# clean columns
por_nearby_venues.columns = [col.split(".")[-1] for col in por_nearby_venues.columns]

por_nearby_venues.head(15)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng,distance
0,Powell's City of Books,Bookstore,45.523180,-122.681375,648
1,Portland Farmer's Market at PSU,Farmers Market,45.512236,-122.684965,1225
2,Guardian Games,Toy / Game Store,45.515521,-122.662038,1084
3,Tasty ’N Alder,American Restaurant,45.521304,-122.683470,732
4,Nong’s Khao Man Gai,Thai Restaurant,45.522323,-122.659365,1179
5,Mediterranean Exploration Company,Mediterranean Restaurant,45.525492,-122.684429,988
6,Washington Park,Park,45.517835,-122.705784,2478
7,Lechon,Latin American Restaurant,45.521303,-122.670750,293
8,Salt & Straw,Ice Cream Shop,45.528929,-122.698298,2113
9,Providence Park,Soccer Stadium,45.521548,-122.691772,1378


In [17]:
sea_venues = sea_results['response']['groups'][0]['items']
    
sea_nearby_venues = json_normalize(sea_venues) # flatten JSON

# filter columns
sea_nearby_venues = sea_nearby_venues.loc[:, filtered_columns]

# filter the category for each row
sea_nearby_venues['venue.categories'] = sea_nearby_venues.apply(get_category_type, axis=1)

# clean columns
sea_nearby_venues.columns = [col.split(".")[-1] for col in sea_nearby_venues.columns]

sea_nearby_venues.head(15)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng,distance
0,Elliott Bay Book Company,Bookstore,47.614689,-122.319885,1429
1,Biscuit B*tch,Breakfast Spot,47.603237,-122.332010,160
2,Pike Place Market,Market,47.609467,-122.341465,1061
3,Seattle Bouldering Project,Climbing Gym,47.593505,-122.310925,1839
4,Pike Place Fish Market,Fish Market,47.608813,-122.340371,951
5,Beecher's Handmade Cheese,Cheese Shop,47.609570,-122.341851,1091
6,REI,Sporting Goods Shop,47.620344,-122.329754,1838
7,CenturyLink Field,Stadium,47.594789,-122.331904,1016
8,Elm Coffee Roasters,Coffee Shop,47.600152,-122.330944,414
9,Seattle Center Playground,Playground,47.621869,-122.349254,2470


#### Group  venue categories at each city

In [18]:
la_nearby_venues['categories'].value_counts()

Trail                           5
Park                            5
Hotel                           5
Art Museum                      4
Farmers Market                  4
Grocery Store                   4
Sandwich Place                  4
American Restaurant             4
Ice Cream Shop                  3
Scenic Lookout                  3
Deli / Bodega                   3
Yoga Studio                     3
Italian Restaurant              3
Coffee Shop                     3
Bakery                          2
Garden                          2
Bookstore                       2
Art Gallery                     2
Wine Shop                       2
Theater                         2
Pizza Place                     2
Cuban Restaurant                1
French Restaurant               1
Wine Bar                        1
Harbor / Marina                 1
Mediterranean Restaurant        1
Theme Park                      1
Observatory                     1
Gym                             1
Fountain      

In [19]:
sf_nearby_venues['categories'].value_counts()

Park                       16
Bakery                      5
Coffee Shop                 4
Ice Cream Shop              3
Bookstore                   3
Science Museum              2
Wine Shop                   2
Gym / Fitness Center        2
Market                      2
Sushi Restaurant            2
Art Museum                  2
Garden                      2
French Restaurant           2
Café                        2
Yoga Studio                 2
Trail                       2
Marijuana Dispensary        2
Grocery Store               2
Climbing Gym                2
Pizza Place                 2
Brewery                     2
Food Truck                  1
Theater                     1
Bar                         1
Adult Boutique              1
New American Restaurant     1
Boxing Gym                  1
Street Food Gathering       1
Indie Movie Theater         1
Hill                        1
Gym                         1
Baseball Stadium            1
Scenic Lookout              1
Souvlaki S

In [20]:
por_nearby_venues['categories'].value_counts()

Park                         10
Grocery Store                 8
Coffee Shop                   7
Brewery                       6
Bakery                        4
Ice Cream Shop                3
Trail                         3
Farmers Market                3
Pizza Place                   3
Sandwich Place                2
Breakfast Spot                2
New American Restaurant       2
Pet Store                     2
Sushi Restaurant              2
Seafood Restaurant            2
Garden                        2
Mediterranean Restaurant      1
Restaurant                    1
Soccer Stadium                1
Toy / Game Store              1
Beer Bar                      1
Boxing Gym                    1
Thai Restaurant               1
Food & Drink Shop             1
Hotel                         1
Russian Restaurant            1
Italian Restaurant            1
Bookstore                     1
Latin American Restaurant     1
State / Provincial Park       1
Steakhouse                    1
Deli / B

In [21]:
sea_nearby_venues['categories'].value_counts()

Park                     13
Bakery                    5
Brewery                   5
Coffee Shop               4
Ice Cream Shop            3
Beach                     3
Japanese Restaurant       3
Playground                2
Cocktail Bar              2
Bookstore                 2
Poke Place                2
Vietnamese Restaurant     2
Scenic Lookout            2
Yoga Studio               2
Taco Place                2
Museum                    2
French Restaurant         2
Farmers Market            2
Gym / Fitness Center      1
Record Shop               1
Wine Bar                  1
Caribbean Restaurant      1
Dumpling Restaurant       1
Fish Market               1
Beer Bar                  1
Bar                       1
Market                    1
Hotel                     1
Trail                     1
Plaza                     1
Gym                       1
Fountain                  1
Baseball Stadium          1
Donut Shop                1
Supermarket               1
Sporting Goods Shop 

#### Narrow venue categories into 6 major categories based on grouping results above

In [22]:
Shopping = ['Arts & Crafts Store','Music Venue','Antique Shop','Boutique','Cosmetics Shop', "Women's Store",'Souvenir Shop','Gift Shop','Toy / Game Store','Shopping Mall','Sporting Goods Shop','Supermarket','Liquor Store','Clothing Store','Beer Store']
Food = ['Sandwich Place','Bakery','Burrito Place','Steakhouse','Seafood Restaurant','Japanese Restaurant','Thai Restaurant', 'Restaurant','Hot Dog Joint','Indian Restaurant','Fried Chicken Joint','Dessert Shop','Pizza Place','Food Truck','Coffee Shop','Hookah Bar','Malay Restaurant','Korean Restaurant','Vietnamese Restaurant','Filipino Restaurant','Arepa Restaurant','American Restaurant','Cajun / Creole Restaurant','Asian Restaurant','African Restaurant','Middle Eastern Restaurant','Russian Restaurant','German Restaurant','Mexican Restaurant','Brazilian Restaurant','Greek Restaurant','Café','French Restaurant','Italian Restaurant','Bagel Shop','Bed & Breakfast','Breakfast Spot','Chinese Restaurant', 'Fast Food Restaurant', 'Donut Shop','Restaurant','Bar', 'Wine Bar','Cocktail Bar','Farmers Market','Diner','Fish Market','Bubble Tea Shop','BBQ Joint','Sushi Restaurant','Frozen Yogurt Shop','Ice Cream Shop','Deli / Bodega','Mediterranean Restaurant','Food Truck','Smoothie Shop','Chocolate Shop']
Entertainment = ['Surf Spot','Theater','Movie Theater','Auditorium','Comedy Club','Strip Club','Tour Provider','Bookstore','Bike Rental / Bike Share','Pool','Tourist Information Center','Golf Course','Spa','Health & Beauty Service','General Entertainment','Gym / Fitness Center','Concert Hall','Zoo','Trail','Gym','Mini Golf','Theme Park']
Museum = ['Public Art','Museum','Art Museum','Performing Arts Venue','Art Gallery','Science Museum']
Scenic = ['Park','Sculpture Garden','Garden Center','Animal Shelter','Beer Garden','Scenic Lookout','Beach','River', 'Bridge','Lake','Monument / Landmark','Hill','Botanical Garden','Fountain','Waterfront','Construction & Landscaping','Observatory','Harbor / Marina','Historic Site','Outdoor Sculpture ']
Hotel = ['Plaza','Hotel']


In [23]:
all_cat = Shopping + Food + Entertainment + Museum + Scenic + Hotel
all_cat_list = [Shopping,Food,Entertainment,Museum,Scenic,Hotel]
all_cat_names = ['Shopping', 'Food','Entertainment', 'Museum', 'Scenic', 'Hotel']

all_nearyby_venues = [la_nearby_venues,sf_nearby_venues,por_nearby_venues,sea_nearby_venues]
cats_count = [0,0,0,0]
eachcat_count = [0,0,0,0]

for i, f in enumerate(all_nearyby_venues):
    for j in range(len(f)):
        if f.iloc[j]['categories'] in all_cat:
            cats_count[i]+=1

cities = ['Los Angeles','San Francisco', 'Portland', 'Seattle']
df_count = pd.DataFrame(cities, columns = ['City'])            
df_count['Total Count of Categories'] = cats_count

for c, cat in enumerate(all_cat_list):
    for i, f in enumerate(all_nearyby_venues):
        for j in range(len(f)):
            if f.iloc[j]['categories'] in cat:
                eachcat_count[i]+=1
    df_count[all_cat_names[c]] = eachcat_count
df_count

,City,Total Count of Categories,Shopping,Food,Entertainment,Museum,Scenic,Hotel
0,Los Angeles,78,3,40,52,59,73,78
1,San Francisco,70,1,32,43,49,69,70
2,Portland,63,4,44,48,50,62,63
3,Seattle,73,3,40,46,49,71,73


#### Visualizing places in 6 different colors representing 6 major categories in each city

Colors of 6 major categories:
- Shopping: Red
- Food: Green
- Entertainment: Purple
- Museum: Lightgray
- Scenic: Blue
- Hotel: Yellow

In [28]:
map_la = folium.Map(location=[la_latitude,la_longitude], zoom_start=11)

color = ''
for lat, lng, label in zip(la_nearby_venues['lat'],la_nearby_venues['lng'],la_nearby_venues['categories']):
    if label in Shopping:
        color = 'red'
    elif label in Food:
        color = 'green'
    elif label in Entertainment:
        color = 'purple'
    elif label in Museum:
        color = 'lightgray'
    elif label in Scenic:
        color = 'blue'
    elif label in Hotel:
        color = 'yellow'
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color=color,
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(map_la)

map_la

In [29]:
map_sf = folium.Map(location=[sf_latitude,sf_longitude], zoom_start=11)

color = ''
for lat, lng, label in zip(sf_nearby_venues['lat'],sf_nearby_venues['lng'],sf_nearby_venues['categories']):
    if label in Shopping:
        color = 'red'
    elif label in Food:
        color = 'green'
    elif label in Entertainment:
        color = 'purple'
    elif label in Museum:
        color = 'lightgray'
    elif label in Scenic:
        color = 'blue'
    elif label in Hotel:
        color = 'yellow'
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color=color,
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(map_sf)

map_sf

In [30]:
map_por = folium.Map(location=[por_latitude,por_longitude], zoom_start=11)

color = ''
for lat, lng, label in zip(por_nearby_venues['lat'],por_nearby_venues['lng'],por_nearby_venues['categories']):
    if label in Shopping:
        color = 'red'
    elif label in Food:
        color = 'green'
    elif label in Entertainment:
        color = 'purple'
    elif label in Museum:
        color = 'lightgray'
    elif label in Scenic:
        color = 'blue'
    elif label in Hotel:
        color = 'yellow'
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color=color,
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(map_por)

map_por

In [31]:
map_sea = folium.Map(location=[sea_latitude,sea_longitude], zoom_start=11)

color = ''
for lat, lng, label in zip(sea_nearby_venues['lat'],sea_nearby_venues['lng'],sea_nearby_venues['categories']):
    if label in Shopping:
        color = 'red'
    elif label in Food:
        color = 'green'
    elif label in Entertainment:
        color = 'purple'
    elif label in Museum:
        color = 'lightgray'
    elif label in Scenic:
        color = 'blue'
    elif label in Hotel:
        color = 'yellow'
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color=color,
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(map_sea)

map_sea